Translating the R sequences from https://arxiv.org/abs/1611.09477 into Python vtreat https://github.com/WinVector/pyvtreat

R original

Python translation

In [1]:
import pandas
import vtreat # https://github.com/WinVector/pyvtreat


freetrade = pandas.read_csv('freetrade.csv')
freetrade.head()

,year,country,tariff,polity,pop,gdp.pc,intresmi,signed,fiveop,usheg
0,1981,SriLanka,NaN,6.0,14988000,461.023590,1.937347,0.0,12.4,0.259311
1,1982,SriLanka,NaN,5.0,15189000,473.763397,1.964430,0.0,12.5,0.255801
2,1983,SriLanka,41.299999,5.0,15417000,489.226562,1.663936,1.0,12.3,0.265502
3,1984,SriLanka,NaN,5.0,15599000,508.173920,2.797462,0.0,12.3,0.298801
4,1985,SriLanka,31.000000,5.0,15837000,525.560852,2.259116,0.0,12.3,0.295243


In [2]:
freetrade.dtypes

year          int64
country      object
tariff      float64
polity      float64
pop           int64
gdp.pc      float64
intresmi    float64
signed      float64
fiveop      float64
usheg       float64
dtype: object